In [ ]:
import tensorflow as tf
from numpy.random import seed
seed(42)
tf.random.set_seed(42)

In [1]:
import os
import numpy as np
import pandas as pd
import nibabel as nib
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from skimage.transform import resize
from scipy.ndimage import zoom
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.losses import *
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.initializers import *
from keras import metrics
from keras import optimizers
import datetime
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import to_categorical
from keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
%matplotlib inline

Using TensorFlow backend.
C:\Users\flori\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\flori\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\flori\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\flori\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [ ]:
from evaluation_metrics import change_to_3D_MRI, calculate_dice, calculate_metrics, calculate_stats_3D_data

## Load the dataset

In [ ]:
def load_3D_data(folder_path = '/content/drive/My Drive/Colab Notebooks/MICCAI_BraTS_2019_Data_Training/', img_scale = 0.266):
  
    tumor_type = ['HGG', 'LGG']

    flair_data = []
    t1_data = []
    t1ce_data = []
    t2_data = []
    seg_data = []
    labels = []


    for t in tumor_type:
        path = folder_path + t
        for i, q in zip(os.listdir(path), tqdm(range(len(os.listdir(path)) -1))):
            new_paths = os.path.join(path, i)
            new_paths_list = os.listdir(new_paths)

            temp_flair_data = []
            temp_t1_data = []
            temp_t1ce_data = []
            temp_t2_data = []
            temp_seg_data = []

            if t == 'HGG':
            labels.append(0)
            if t == 'LGG':
            labels.append(1)

            for j in new_paths_list:
            file_path = os.path.join(new_paths, j)
            if file_path[-10:-7] == 'air':
                temp_img = nib.load(file_path)
                temp_img_array = np.asarray(temp_img.dataobj)
                temp_img_array_r = np.asarray(zoom(temp_img_array, (img_scale, img_scale, img_scale), prefilter=False))
                temp_flair_data.append(temp_img_array_r)

            if file_path[-10:-7] == '_t1':
                temp_img = nib.load(file_path)
                temp_img_array = np.asarray(temp_img.dataobj)
                temp_img_array_r = np.asarray(zoom(temp_img_array, (img_scale, img_scale, img_scale), prefilter=False))
                temp_t1_data.append(temp_img_array_r)

            if file_path[-10:-7] == '1ce':
                temp_img = nib.load(file_path)
                temp_img_array = np.asarray(temp_img.dataobj)
                temp_img_array_r = np.asarray(zoom(temp_img_array, (img_scale, img_scale, img_scale), prefilter=False))
                temp_t1ce_data.append(temp_img_array_r)

            if file_path[-10:-7] == '_t2':
                temp_img = nib.load(file_path)
                temp_img_array = np.asarray(temp_img.dataobj)
                temp_img_array_r = np.asarray(zoom(temp_img_array, (img_scale, img_scale, img_scale), prefilter=False))
                temp_t2_data.append(temp_img_array_r)

            if file_path[-10:-7] == 'seg':
                temp_img = nib.load(file_path)
                temp_img_array = np.asarray(temp_img.dataobj)
                temp_img_array_r = np.asarray(zoom(temp_img_array, (img_scale, img_scale, img_scale), prefilter=False))
                temp_seg_data.append(temp_img_array_r)

            temp_flair_data_arr = np.asarray(temp_flair_data)
            flair_data.append(temp_flair_data_arr)

            temp_t1_data_arr = np.asarray(temp_t1_data)
            t1_data.append(temp_t1_data_arr)

            temp_t1ce_data_arr = np.asarray(temp_t1ce_data)
            t1ce_data.append(temp_t1ce_data_arr)

            temp_t2_data_arr = np.asarray(temp_t2_data)
            t2_data.append(temp_t2_data_arr)

            temp_seg_data_arr = np.asarray(temp_seg_data)
            seg_data.append(temp_seg_data_arr)

            print('', end='\r')

            flair_data_arr = np.asarray(flair_data)
            t1_data_arr = np.asarray(t1_data)
            t1ce_data_arr = np.asarray(t1ce_data)
            t2_data_arr = np.asarray(t2_data)
            seg_data_arr = np.asarray(seg_data)
            labels_arr = np.asarray(labels)
  
  return flair_data_arr, t1_data_arr, t1ce_data_arr, t2_data_arr, seg_data_arr, labels_arr

In [ ]:
flair, t1, t1ce, t2, seg, labels = load_3D_data()

print(flair.shape)
print(t1.shape)
print(t1ce.shape)
print(t2.shape)
print(seg.shape)
print(labels.shape)

## combine the 4 MRI types

In [ ]:
def combine_data(x1, x2, x3, x4):
    data_comb = []
    for i in range(len(x1)):
        combined_temp = []
        combined_temp.append(np.asarray(x1[i][0][:,:,:]))
        combined_temp.append(np.asarray(x2[i][0][:,:,:]))
        combined_temp.append(np.asarray(x3[i][0][:,:,:]))
        combined_temp.append(np.asarray(x4[i][0][:,:,:]))
        combined_temp_arr = np.asarray(combined_temp)
        data_comb.append(combined_temp_arr)
    return np.asarray(data_comb)

In [ ]:
combined_data = combine_data(flair,t1,t1ce,t2)
print(combined_data.shape)

## normalize the data

In [ ]:
def normalize_data(data):
    norm_data = []

    for i in data:
        temp_data = []
        for j in i:
            new_data = j/np.max(j)
            temp_data.append(new_data)
        temp_data = np.asarray(temp_data)
        norm_data.append(temp_data)

    return np.asarray(norm_data)

In [ ]:
normalized_data = normalize_data(combined_data)
print(normalized_data.shape)

## Split the data (and add labels)

In [ ]:
def label_data(data, label):
    labeled_data = []

    for i, j in zip(label, data):
        temp = []
        temp.append(i)
        temp.append(j)
        temp_arr = np.asarray(temp)
        labeled_data.append(temp_arr)
    return np.asarray(labeled_data)

In [ ]:
labeled_data = label_data(normalized_data, labels)

In [ ]:
X_train, X_test_val, y_train, y_test_val = train_test_split(labeled_data, seg, test_size=0.30, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, test_size=0.50, random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
def split_labels(l_data):
    labels = []
    data = []

    for i in l_data:
        labels.append(i[0])
        data.append(i[1])

    return np.asarray(labels), np.asarray(data)

In [ ]:
X_train_labels, X_train = split_labels(X_train)
X_val_labels, X_val = split_labels(X_val)
X_test_labels, X_test = split_labels(X_test)

## Change the data into 2D slices

In [ ]:
# for train
def change_to_2D_RGB(x):
    rgb_data = []

    for i in range(x.shape[0]):
        for z in range(x.shape[-1]):
            rgb_temp = []
            for y in range(x.shape[1]):
                rgb_temp.append(np.asarray(x[i][y][:,:,z]))
                rgb_temp_arr = np.asarray(rgb_temp)
        rgb_data.append(rgb_temp_arr)

    rgb_data_arr = np.asarray(rgb_data)
    return rgb_data_arr

In [ ]:
# for test
def change_to_2D_slices(data_3D):
    data_slices_2D = []

    for i in range(len(data_3D)):
        for z in range(data_3D[i].shape[-1]):
            slice_2D = np.asarray(data_3D[i][:,:,:,z])
            data_slices_2D.append(slice_2D)

    data_slices_2D_arr = np.asarray(data_slices_2D)
    return data_slices_2D_arr

In [ ]:
def change_2D_bi(y_data):
    return np.where(y_data == 0, y_data, 1)

In [ ]:
X_train = change_to_2D_RGB(X_train)
X_val = change_to_2D_RGB(X_val)
X_test = change_to_2D_RGB(X_test)

y_train = change_2D_bi(change_to_2D_slices(y_train))
y_val = change_2D_bi(change_to_2D_slices(y_val))
y_test = change_2D_bi(change_to_2D_slices(y_test))


print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

## Load the U-Net to make a sequence

In [5]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    coef = (2. * intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return coef

In [6]:
def loss(y_true, y_pred):
    def dice_coef_loss(y_true, y_pred):
        return 1-dice_coef(y_true, y_pred)

    return binary_crossentropy(y_true, y_pred) + dice_coef_loss(y_true, y_pred)

In [ ]:
import tensorflow as tf
import keras.backend.tensorflow_backend as tfback

print("tf.__version__ is", tf.__version__)
print("tf.keras.__version__ is:", tf.keras.__version__)

def _get_available_gpus():
    """Get a list of available gpu devices (formatted as strings).

    # Returns
        A list of available GPU devices.
    """
    #global _LOCAL_DEVICES
    if tfback._LOCAL_DEVICES is None:
        devices = tf.config.list_logical_devices()
        tfback._LOCAL_DEVICES = [x.name for x in devices]
    return [x for x in tfback._LOCAL_DEVICES if 'device:gpu' in x.lower()]

tfback._get_available_gpus = _get_available_gpus

In [7]:
input_size = (4,64,64)


inputs = Input(input_size)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(inputs)
conv1 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv1)
batch1 = BatchNormalization()(conv1)
pool1 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch1)

conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool1)
conv2 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv2)
batch2 = BatchNormalization()(conv2)
pool2 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch2)

conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool2)
conv3 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv3)
batch3 = BatchNormalization()(conv3)
pool3 = MaxPooling2D(pool_size=(2, 2), data_format = 'channels_first')(batch3)

conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(pool3)
conv4 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv4)
batch4 = BatchNormalization()(conv4)
drop4 = Dropout(0.5)(batch4)

up5 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(drop4))
merge5 = concatenate([conv3,up5], axis = 1)
conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge5)
conv5 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv5)
batch5 = BatchNormalization()(conv5)

up6 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch5))
merge6 = concatenate([conv2,up6], axis = 1)
conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge6)
conv6 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv6)
batch6 = BatchNormalization()(conv6)

up7 = Conv2D(32, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(UpSampling2D(size = (2,2), data_format = 'channels_first')(batch6))
merge7 = concatenate([conv1,up7], axis = 1)
conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(merge7)
conv7 = Conv2D(32, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal', data_format = 'channels_first')(conv7)
conv8 = Conv2D(1, 1, activation = 'sigmoid', data_format = 'channels_first')(conv7)



model = Model(input = inputs, output = conv8)

model.compile(optimizer = Adam(lr = 0.001), loss = loss , metrics = [dice_coef])
    


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


C:\Users\flori\Anaconda3\lib\site-packages\ipykernel_launcher.py:45: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


## U-net test results

In [ ]:
model.load_weights('unet_weights.35-0.15.hdf5')

In [ ]:
test_result = model.predict([X_test,test_sequence])
test_result_3D = change_to_3D_MRI(test_result)
y_test_3D = change_to_3D_MRI(y_test)
test_scores = calculate_stats_3D_data(test_result_3D, y_test_3D, labels_X_test)

In [ ]:
test_scores

## Create the sequence

In [ ]:
# the best model loaded from the U-Net
model.load_weights('unet_weights.35-0.15.hdf5')

# n = the length of the sequence used in the RFCN
n = 6

def create_sequence(X_data, n=n, y_data = y_train, y_true = 'N'):

    test_masks = model.predict(X_data)
    test_masks = np.round(test_masks)

    r = np.arange(41)
    x =  np.tile(r, int(len(test_masks)/41))
    y = np.arange(len(test_masks))

    test_sequence = []

    for i, j in zip(x, y):
        if i == 0:
            base = np.zeros((n,64,64))     
            base[0] = 2
            test_sequence.append(base)

        else:
            if y_true == 'Y':
                temp = y_data[(j-1)][:,:,:] 
                temp = np.where(temp == 1, 10, 5)
                base_2 = test_sequence[j-1].copy()
                base_plus = np.vstack((temp, base_2))[:n]
                test_sequence.append(base_plus)

            else:
                temp = test_masks[(j-1)][:,:,:] 
                temp = np.where(temp == 1, 10, 5)
                base_2 = test_sequence[j-1].copy()
                base_plus = np.vstack((temp, base_2))[:n]
                test_sequence.append(base_plus)

    seq_arr = np.asarray(test_sequence)

    flipped_sequence = []
    for i in seq_arr:
        flipped = np.flip(i, axis = 0)
        flipped_sequence.append(flipped)

    return np.expand_dims(np.asarray(flipped_sequence), axis = 4)

In [ ]:
train_sequence = create_sequence(X_train, y_data = y_train, y_true = 'N')
val_sequence = create_sequence(X_val, y_data = y_val, y_true = 'N')
test_sequence = create_sequence(X_test, y_data = y_test, y_true = 'N')

print(train_sequence.shape)
print(val_sequence.shape)
print(test_sequence.shape)

In [ ]:
# test if the sequence is correct
plt.imshow(y_train[16][0,:,:])

In [ ]:
plt.imshow(train_sequence[17].reshape((n,64,64))[(n-1)])